In [13]:
import pandas as pd
from molsol import KDESol, KDESolConfig
import json
import selfies as sf
import tensorflow as tf

with open("molsol/voc.json", 'r') as inp:
  voc = json.load(inp)

m = KDESol(KDESolConfig(), './molsol/kde10_lstm_r')

In [14]:
m.model

In [59]:
AqSolDB = pd.read_csv(f"curated-solubility-dataset.tab",sep="\t")[['SMILES', 'Solubility']]
labels = AqSolDB["Solubility"]
AqSolDB["SELFIES"] = AqSolDB["SMILES"].map(m.encoder)
AqSolDB = AqSolDB.dropna()
AqSolDB["SELFIES_TOKENS"] = AqSolDB["SELFIES"].map(m.stoi)
AqSolDB = AqSolDB.dropna()

padded_seqs = tf.keras.preprocessing.sequence.pad_sequences(AqSolDB["SELFIES_TOKENS"].to_numpy(), padding="post")
sol_values = AqSolDB["Solubility"].to_numpy()

In [60]:
split=int(0.2*len(padded_seqs))
val_data = tf.data.Dataset.from_tensor_slices(
        (padded_seqs[split:2*split], sol_values[split:2*split])
    )
val_data = val_data.batch(32).cache().prefetch(
        tf.data.AUTOTUNE
    )

In [61]:
import numpy as np
X_val, y_val = (np.concatenate([x[0].numpy()[np.newaxis,...] for x in val_data.unbatch()]), 
                np.concatenate([y for x, y in val_data]))

In [62]:
X_val.shape

(1984, 634)

In [73]:
import numpy as np

slf = m.encoder("COCOOCOCO")
tok = np.array(m.stoi(slf))
# padded_tok = tf.keras.preprocessing.sequence.pad_sequences([tok,tok], padding="post")

# print(X_val[0])
# print(tok[0])
# pred = m(X_val)
pred = m(np.array([tok, tok]))


1/1 [==============================] - 0s 238ms/step


In [68]:
pred

array([[0.12362216, 1.7795107 , 0.4629264 ],
       [0.12362216, 1.7795107 , 0.4629264 ],
       [0.12362216, 1.7795107 , 0.4629264 ]], dtype=float32)